# <u>Lesson 1</u>: what makes code readable ?

### Task: take the union of two meshes.
### Given two sets of elements and their corresponding points,
### create a union mesh from the two input meshes without duplicate elements and points.

In [ ]:
# make two triangular meshes characterised by their element indices and points

import numpy as np

elems0 = np.array([ [0, 1, 4],
                    [4, 1, 5],
                    [1, 2, 5],
                    [5, 2, 6],
                    [2, 3, 6],
                    [6, 3, 7],
                    [4, 5, 8],
                    [8, 5, 9],
                    [5, 6, 9],
                    [9, 6, 10],
                    [6, 7, 10],
                    [10, 7, 11] ])

points0 = np.stack(list(map(np.ravel, np.meshgrid( np.linspace(0, 3, 4),
                                                   np.linspace(0, 2, 3) ))), axis=1)


# same mesh but shifted by +2 in the x direction
elems1 = elems0
points1 = points0 + np.array([[2, 0]])

In [ ]:
from matplotlib import pyplot as plt

def plot_meshes(list_of_elements, list_of_points):

    fig, ax = plt.subplots()
    for elems, points in zip(list_of_elements, list_of_points):
        ax.triplot(*points.T, elems, alpha=0.5)

    plt.show()

### we plot the two meshes we just created

In [ ]:
plot_meshes([elems0, elems1], [points0, points1])

<hr style="border:1px solid blue">

### 

### The absolute bloody beginner solution

#### (seeing it breaks my heart)

In [ ]:
elements = [elems0, elems1]
points = [points0, points1]

map_point_index = {}
index = 0
new_elements = []
new_points = []
seen = set()

for i in range(2):
    myelems = elements[i]
    mypoints = points[i]
    my_new_elems = []
    for j in range(len(myelems)):
        my_new_elem = []
        myelement = myelems[j]
        for k in range(len(myelement)):
            myindex = myelement[k]
            mypoint = tuple(mypoints[myindex])
            if mypoint not in map_point_index:
                map_point_index[mypoint] = index
                index += 1
                new_points.append(mypoint)
            my_new_elem.append(map_point_index[mypoint])
        my_identifier = tuple(sorted(my_new_elem))
        if my_identifier not in seen:
            my_new_elems.append(my_new_elem)
            seen.add(my_identifier)
    new_elements.append(my_new_elems)

new_elements = np.concatenate(new_elements)
new_points = np.array(new_points)

plot_meshes([new_elements], [new_points])

print('Number of points: ', len(new_points))
print('Number of elements: ', len(new_elements))

### 

### The somewhat more idiomatic solution

#### (still not a good code)

In [ ]:
map_point_index = {}
index = 0
new_elements = []
new_points = []
seen = set()

for myelems, mypoints in zip([elems0, elems1], [points0, points1]):
    my_new_elems = []
    for myelement in myelems:
        my_new_elem = []
        for myindex in myelement:
            mypoint = tuple(mypoints[myindex])
            if mypoint not in map_point_index:
                map_point_index[mypoint] = index
                index += 1
                new_points.append(mypoint)
            my_new_elem.append(map_point_index[mypoint])
        my_identifier = tuple(sorted(my_new_elem))
        if my_identifier not in seen:
            my_new_elems.append(my_new_elem)
            seen.add(my_identifier)
    new_elements.append(my_new_elems)

new_elements = np.concatenate(new_elements)
new_points = np.array(new_points)

plot_meshes([new_elements], [new_points])

print('Number of points: ', len(new_points))
print('Number of elements: ', len(new_elements))

### 

### A decent solution

#### (Close to optimal without using numpy functionality)

In [ ]:
from itertools import count
from collections import defaultdict

map_point_index = defaultdict(count().__next__)
seen = set()

new_elems = []
for elems, points in zip([elems0, elems1], [points0, points1]):
    for elem in elems:
        new_elem = [map_point_index[point] for point in map(tuple, points[elem])]
        elem_identifier = tuple(sorted(new_elem))
        if elem_identifier not in seen:
            new_elems.append(new_elem)
            seen.add(elem_identifier)

new_elems = np.array(new_elems)
new_points = np.stack(list(map_point_index.keys()))

plot_meshes([new_elems], [new_points])

print('Number of points: ', len(new_points))
print('Number of elements: ', len(new_elems))

### 

### Good non-numpy solution (similar to the previous one)

In [ ]:
from itertools import count
from collections import defaultdict

map_point_index = defaultdict(count().__next__)
seen = set()

new_elems = []
for elems, points in zip([elems0, elems1], [points0, points1]):
    for elem in elems:
        new_elem = [map_point_index[point] for point in map(tuple, points[elem])]
        if ( elem_identifier := tuple(sorted(new_elem)) ) not in seen:
            new_elems.append(new_elem)
            seen.add(elem_identifier)

new_elems = np.array(new_elems)
new_points = np.stack(list(map_point_index.keys()))

plot_meshes([new_elems], [new_points])

print('Number of points: ', len(new_points))
print('Number of elements: ', len(new_elems))

### 

### A fancy but somewhat overengineered solution (matter of taste)

In [ ]:
from itertools import count
from collections import defaultdict

map_point_index = defaultdict(count().__next__)
seen = set()

new_elems = []
for elems, points in zip([elems0, elems1], [points0, points1]):
    for elem in elems:
        new_elem = [map_point_index[point] for point in map(tuple, points[elem])]
        (identifier := tuple(sorted(new_elem))) in seen or seen.add(identifier) or new_elems.append(new_elem)

new_elems = np.array(new_elems)
new_points = np.stack(list(map_point_index.keys()))

plot_meshes([new_elems], [new_points])

print('Number of points: ', len(new_points))
print('Number of elements: ', len(new_elems))

### 
### The numpy solution 
#### (achieves the best readability provided comments are added, a little less memory efficient) 

In [ ]:
from itertools import count

# get all unique points of the two sets of points
new_points = np.unique(np.concatenate([points0, points1]), axis=0)

# map each unique point to an index
map_point_index = dict(zip(map(tuple, new_points), count()))

# map both meshes' elements' points to the new index
mapped_elems = np.apply_along_axis(lambda x: map_point_index[tuple(x)],
                                   axis=-1,
                                   arr=np.concatenate([points0[elems0], points1[elems1]]))

# find the indices of the first occurence of the transformed elements that are unique
_, unique_indices = np.unique(np.sort(mapped_elems, axis=1), return_index=True, axis=0)

# keep only the unique occurences
new_elems = mapped_elems[unique_indices]

plot_meshes([new_elems], [new_points])

print('Number of points: ', len(new_points))
print('Number of elements: ', len(new_elems))

### 
## What makes python code readable ?

### What we know so far:

* Using idioms (zip, enumerate, itertools, collections, comprehensions, := ) to reduce boilerplate.
* Fewer (but not too few) lines of code.
* Using => => => indentation instead of => <= => <=.
* Descriptive variable names.
* A program flow that almost reads like english.
* Comments and using library functionality (for instance numpy) where possible.

<hr style="border:1px solid blue">

### 
## <u>Some open questions</u>

### 
### <u>Open question</u>: which one is more pythonic ?

In [ ]:
def _first_order_derivative(func):
    # return derivative
    pass

# version 0 or version 1 ?

def nth_derivative_v0(func, n=1):
    assert (n := int(n)) >= 0
    if n == 0:
        return func
    else:
        return nth_derivative_v0(_first_order_derivative(func), n=n-1)
    

def nth_derivative_v1(func, n=1):
    assert (n := int(n)) >= 0
    if n == 0:
        return func
    return nth_derivative_v1(_first_order_derivative(func), n=n-1)

### 

### <u>Open question</u>: which meshgrid call signature do you find more readable ?

In [ ]:
import numpy as np

### forget about this part
def make_meshgrid0(dims):
    return np.stack(np.meshgrid(*map(np.arange, dims)), axis=-1).reshape(-1, len(dims))

def make_meshgrid1(*dims):
    return np.stack(np.meshgrid(*map(np.arange, dims)), axis=-1).reshape(-1, len(dims))
###


# THIS PART
# option a or b ?
a = make_meshgrid0([3, 2, 2])

b = make_meshgrid1(3, 2, 2)

# check if they're really the same
print('a equals b: ', (a == b).all())

# both generalise to arbitrary dimensions
a = make_meshgrid0([3, 2])

b = make_meshgrid1(3, 2, 2, 4)

### 
### <u>Open question</u>: which one do you think is more pythonic ?

In [ ]:
# Find the greatest divisor of an integer (excluding self) and return it.
# Also, print all other (smaller) divisors, if any.

def greatest_divisor_v0(val):
    """ excluding `val` itself """
    *other_divisors, greatest_divisor = [i for i in range(1, val) if val % i == 0]
    if other_divisors:
        print("For {}, I also found the divisors {}.".format(val, other_divisors))
    return greatest_divisor


def greatest_divisor_v1(val):
    """ excluding `val` itself """
    divisors = [i for i in range(1, val) if val % i == 0]
    divisors, greatest_divisor = divisors[:-1], divisors[-1]
    if len(divisors) > 0:
        print("For {}, I also found the divisors {}.".format(val, divisors))
    return greatest_divisor


print('Greatest divisor of 10: ', greatest_divisor_v0(10), '\n')
print('Greatest divisor of 10: ', greatest_divisor_v1(10))

### 
## What makes a code more pythonic ?
##  (usually, that means more readable)

### <u>Some additional insights</u>:

* Early handling of special cases, avoiding if-else indentation when redundant (example 1).
* Avoiding nested parentheses: `f(a, b, c)` instead of `f([a, b, c])` (example 2).
* using star syntax `*args` where possible (examples 2 + 3).
* `if object` rather than `if (object has property)`. For instance `if divisors` instead of `if len(divisors) > 0` (example 3).

<hr style="border:1px solid blue">

### 
### Now that we have an idea of what makes a code pythonic, let's move on to the next

## <u>Lesson 2</u>: `*args, **kwargs`, star syntax variable unpacking

In [ ]:
%reset -f

### 
### What does this dummy function do ?

In [ ]:
# pass whatever arguments into this function and print the input it receives
def dummy_function(*args, **kwargs):
    print('Received the following args and kwargs: \n')
    print('args: ', args)
    print('kwargs: ', kwargs, '\n \n')
    

print('Only positional arguments: \n')
dummy_function(1, 2, 'a', [1, 2, 3])

print('Only keyword arguments: \n')
dummy_function(a=1, b=2, c='a', d=[1, 2, 3])

print('Both positional and keyword arguments: \n')
dummy_function(1, 'a', (5, 6), a=5, b='a', c=[5, 6])

### 
### <u> Observation</u>:
### for a function of the form `f(*args, **kwargs)`:
* all positional arguments passed are inside of the function available as a tuple called `args`
* all keyword arguments passed are available as a dictionary `kwargs` of the form `{str(keyword): passed_value}`

<hr style="border:1px solid blue">

### 
## <u>Exercise 2.1</u>: 
### write a function that takes positional arguments (numbers) and adds them.
* for example:  f(1, 2) = 2; f(4, 5, 6) = 120; f(1, 1, 2, 3) = 6
* by convention: f() = 0

In [ ]:
#def f( ? ? ? ):
    # your code here
    #return  ? ? ?
    
print(f(1, 2))
print(f(4, 5, 6))
print(f(1, 1, 2, 3))

### solution:

In [ ]:
def f(*args):
    # return sum(*args) would work too. sum( . ) accepts both tuples / list and positional arguments.
    return sum(args)

print(f(1, 2))
print(f(4, 5, 6))
print(f(1, 1, 2, 3))

<hr style="border:1px solid blue">

### 
## <u> Exercise 2.2 </u>: 
### write a function that takes only keyword arguments `subject = (value, weight)`
### and computes the weighted average of all passed subjects
### $\text{weighted_average} = \frac{\sum_i \text{value}_i \, \times \, \text{weight}_i}{\sum_i \text{weight}_i}$

#### `weighted_average(math=(80, 4), english=(60, 2), geography=(50, 1)) = (320 + 120 + 50) / (4 + 2 + 1) = 70`

In [ ]:
# def weighted_average(? ? ?):
    # your code here
    # return ???



print(weighted_average(math=(80, 4), english=(60, 2), geography=(50, 1)))

### solution (not very pythonic !!):

In [ ]:
# we'll learn how to write this function better, don't worry.
def weighted_average(**kwargs):
    weighted_values = 0
    weights = 0
    # subject: english
    # values_weights: (60, 2)
    for subject, values_weights in kwargs.items():
        weighted_values += values_weights[0] * values_weights[1]
        weights += values_weights[1]
    return weighted_values / weights

print(weighted_average(math=(80, 4), english=(60, 2), geography=(50, 1)))


# more pythonic solution, maybe too difficult right now
def weighted_average_py(**kwargs):
    return sum(a * b for a, b in kwargs.values()) / sum(b for a, b in kwargs.values())

<hr style="border:1px solid blue">

### 
### The `*args, **kwargs` syntax can also be used in the opposite way.
### Observe the following:

In [ ]:
def compute_area(length, width, unit='meters'):
    area = length * width
    return f"The area is {area} square {unit}."

kwargs = {'unit': 'kilometers'}

compute_area(2, 5, **kwargs)

### 
### We are allowed to drop a `{str(key): value}`  dictionary into a function that accepts keyword arguments via double dereferencing.
#### If the dict's `.keys()` are the same or a subset of the function's (keyword) arguments, it will work.

#### `test = f(a=5, b=2)` is equivalent to `kwargs = {'a': 5, 'b': 2}` and then `test = f(**kwargs)`

<hr style="border:1px solid blue">

### 
## <u> Application: keyword argument forwarding</u>

![scipykwargs](img/scipykwargs.png)

### 
## <u>Exercise 2.3</u>:
### write a function that uses `scipy.optimize.minimize` to find the argmin of $a x^2 + b x + c$.
### The function must be able to forward relevant keyword arguments to `scipy.optimize.minimize`.
### Find the argmin of the quadratic function using `method='SLSQP'`.

### 
### <u>The wrong way</u>:
#### (doing this will result in capital punishment)

In [ ]:
from scipy.optimize import minimize


# uuuuugh, lots of boilerplate >.<
def minimize_quadratic_function(a=1, b=2, c=3, args=(), 
                                               method=None, 
                                               jac=None,
                                               hess=None,
                                               hessp=None,
                                               bounds=None,
                                               constraints=(),
                                               tol=None,
                                               callback=None,
                                               options=None):
    fun = lambda x: a * x ** 2 + b * x + c
    x0 = 0
    return minimize(fun, x0, args=args,
                             method=method,
                             jac=jac,
                             hess=hess,
                             hessp=hessp,
                             bounds=bounds,
                             constraints=constraints,
                             tol=tol,
                             callback=callback,
                             options=options).x[0]

# min 0.5 x**2 + x + 1 is assumed at x = -1
print('The minimum of 0.5 x^2 + x + 1 is assumed at x =',
       minimize_quadratic_function(a=.5, b=1, c=1, method='SLSQP'))

### 
### Implement it correctly:

In [ ]:
from scipy.optimize import minimize

#def minimize_quadratic_function(a=1, b=2, c=3, ????):
    # fun = lambda x: a * x ** 2 + b * x + c
    # x0 = 0
    # return ???

# min 0.5 x**2 + x + 1 is assumed at x = -1
print('The minimum of 0.5 x^2 + x + 1 is assumed at x =',
       minimize_quadratic_function(a=.5, b=1, c=1, method='SLSQP'))

### 
### solution:

In [ ]:
from scipy.optimize import minimize

def minimize_quadratic_function(a=1, b=2, c=3, **scipykwargs):
    fun = lambda x: a * x ** 2 + b * x + c
    x0 = 0
    return minimize(fun, x0, **scipykwargs).x[0]

# min 0.5 x**2 + x + 1 is assumed at x = -1
print('The minimum of 0.5 x^2 + x + 1 is assumed at x =',
       minimize_quadratic_function(a=.5, b=1, c=1, method='SLSQP'))    

<hr style="border:1px solid blue">

### 
### Observe the following:

In [ ]:
def abc_formula(a, b, c):
    "Return the roots of a quadratic equation of the form ax^2 + bx + c"
    discriminant = (b ** 2 - 4 * a * c)**.5
    return (-b - discriminant) / (2 * a), (-b + discriminant) / (2 * a)

def abc_monomic(*args):
    "Same as abc_formula but a = 1."
    # abc_monomic(1, 2) => args = (2, 3)
    # we drop args back into `abc_formula`
    # abc_formula(1, *args) is then the same as abc_formula(1, 2, 3)
    return abc_formula(1, *args)

print('The complex roots of f = x^2 + 3 * x + 1 read are {:.7f} and {:.7f}'.format(*abc_monomic(3, 1)))

### 
### We can drop a list or tuple's values back into a function using star dereferencing
#### `test = f(1, 2, 3)` is equivalent to `args = (1, 2, 3)` and then `test = f(*args)`

#### intuition: the `*` dereferencing ~ *removes the outer parentheses* ~
#### `f(*(1, 2, 3))` is equivalent to `f(1, 2, 3)`
#### same for lists `f(*[1, 2, 3])` => `f(1, 2, 3)`

<hr style="border:1px solid blue">

### 
## <u> Exercise 2.4 </u> :
### Given a function `f(*args)` that sums its pos. arguments.
### Write a function `g(head, tail)` that accepts two tuples / lists of numbers and sums all of them.
* `g([1, 2, 3], (4, 5, 6)) => f(1, 2, 3, 4, 5, 6) = 21` (note that one is a list and one is a tuple)

In [ ]:
def f(*args):
    return sum(args)

#def g(???):
#    ???

print(g([1, 2, 3], (4, 5, 6)))

### solution:

In [ ]:
def f(*args):
    return sum(args)

#we can also just dereference both head and tail
def g(head, tail):
    # head = [1, 2, 3]
    # tail = (4, 5, 6)
    # => f(*head, *tail) = f(*[1, 2, 3], *(4, 5, 6)) = f(1, 2, 3, 4, 5, 6)
    return f(*head, *tail)

print(g([1, 2, 3], (4, 5, 6)))

<hr style="border:1px solid blue">

### 
### Having gained an intuition for `*args, **kwargs`, we are now in the position to learn **star syntax variable unpacking**

### what does this do ?

In [ ]:
tail = (4, 5, 6)

joined_tuple = (1, 2, 3, *tail)

print(joined_tuple)

### => We can drop tuples' / lists' contents into tuples / lists via dereferencing
<hr style="border:1px solid blue">

### 
### <u>Further examples</u>:

In [ ]:
# multiple list / tuple unpacking
print( [1, 2, *[3, 4, 5], 6, 7, *(8, 9, 10)] )

# we can also dereference ranges into tuples / lists
print( (*[1, 2, 3], *range(4, 11)) )

### 
### What do you guys think this does ?

### `(a, *tail) = [1, 2, 3, 4]`
### What is `a` ? what is `tail` ?
### <u>the answer</u>:

In [ ]:
(a, *tail) = [1, 2, 3, 4]

print(a)
print(tail)

### 
### we can forego the outer parentheses (both left and / or right) when no confusion is possible

In [ ]:
a, *tail = 1, 2, 3, 4

print(a)
print(tail)

### 
## <u>Exercise 2.5</u>
### Given a `tuple` of tuples of the form
### `tot = ((a, b, c), (d, e, f))`, 
### unpack `a, d, e, f` into variables of the same name and create a list `bc` containing `b` and `c`
### **IN ONE LINE OF CODE**
### 
### The wrong solution (don't do it like this)

In [ ]:
tot = ((1, 2, 3), (4, 5, 6))

a, d, e, f = tot[0][0], tot[1][0], tot[1][1], tot[1][2]
bc = [tot[0][1], tot[0][2]]

print(a, bc, d, e, f)

### 
### Implement it correctly:

In [ ]:
tot = ((1, 2, 3), (4, 5, 6))

### your code here

print(a, bc, d, e, f)

### 
### solution:

In [ ]:
tot = ((1, 2, 3), (4, 5, 6))

(a, *bc), (d, e, f) = tot

print(a, bc, d, e, f)

<hr style="border:1px solid blue">

### 
### The same kind of unpacking works for numpy arrays

In [ ]:
import numpy as np

# create an array (matrix) of shape A.shape == (2, 3)
A = np.arange(6).reshape(2, 3)

print('A: \n', A, '\n')

# a numpy array can be regarded as a `list of lists`
# we unpack an array along its first axis
# => a, b = A, with A.shape == (2, 3)
# creates two new np.ndarrays of shape (3,)
a, b = A

print('a is of type `{}` and reads: '.format(a.__class__.__name__), a, '\n')
print('b is of type `{}` and reads: '.format(b.__class__.__name__), b, '\n')


# unpacking a and into their consituents again creates numbers (integers in this case)
(a0, a1, a2), (b0, b1, b2) = A
print('a0, a1, a2, b0, b1, b2: ','{}, {}, {}, {}, {}, {}'.format(a0, a1, a2, b0, b1, b2), '\n')

### 
<hr style="border:1px solid blue">

## <u>Application</u>
### Given a ($2$D) triangular element in $\mathbb{R}^3$ characterised by its vertices `A = [a, b, c]` (a, b, c row vectors, counterclockwise)
### compute the surface area of the triangle.

### 

### <u>the wrong solution</u>:

In [ ]:
import numpy as np

def surface_area(A: np.ndarray) -> float:
    # make sure A has the correct shape
    assert A.shape == (3, 3)
    # create triangle's jacobian J = [b - a; c - a] (column vectors)
    a, b, c = A[0, :], A[1, :], A[2, :]
    J = np.stack([b - a, c - a], axis=1)
    
    # create the local metric tensor by taking the outer product
    G = J @ J.T
    
    # unpack G's entries to compute the surface area as 1/2 sqrt(determinant)
    a00, a01, a10, a11 = G[0, 0], G[0, 1], G[1, 0], G[1, 1]
    return .5 * ((a00 * a11 - a01 * a10)**.5)


A = np.array([
              [0, 0, 0],
              [1, 0, 1],
              [0, 1, 0],
             ])

print('The surface area reads: ', surface_area(A))

### 
### <u>The correct solution</u>:

In [ ]:
import numpy as np

def surface_area(A: np.ndarray) -> float:
    # make sure A has the correct shape
    assert A.shape == (3, 3)
    
    # unpack the rows
    a, b, c = A
    
    # create triangle's jacobian J = [b - a; c - a] (column vectors)
    J = np.stack([b - a, c - a], axis=1)
    
    # create the local metric tensor by taking the outer product
    G = J @ J.T
    
    # unpack G's entries to compute the surface area as 1/2 sqrt(determinant)
    (a00, a01), (a10, a11) = G
    return .5 * ((a00 * a11 - a01 * a10)**.5)


A = np.array([
              [0, 0, 0],
              [1, 0, 1],
              [0, 1, 0],
             ])

print('The surface area reads: ', surface_area(A))

### 
### 
### 
### Why did it not work ???

In [ ]:
import numpy as np

# J inside of `surface_area` is of shape J.shape == (3, 2)
J = np.random.randn(3, 2)

print((J @ J.T).shape)

### 
### `G = J @ J.T` has shape (3, 3) which is wrong. It should have been `G = J.T @ J` of shape `(2, 2)`.
### These errors happen easily and go unnoticed for a long long time ...
### Our idiomatic version caught this error because the unpacking implicitly assumed `G` to be of shape `G.shape == (2, 2)`.
### 
### <u>The correct solution</u> (this time for real):

In [ ]:
import numpy as np

def surface_area(A: np.ndarray) -> float:
    # make sure A has the correct shape
    assert A.shape == (3, 3)
    
    # unpack the rows
    a, b, c = A
    
    # create triangle's jacobian J = [b - a; c - a] (column vectors)
    J = np.stack([b - a, c - a], axis=1)
    
    # create the local metric tensor by taking the outer product
    G = J.T @ J
    
    # unpack G's entries to compute the surface area as 1/2 sqrt(determinant)
    (a00, a01), (a10, a11) = G
    return .5 * ((a00 * a11 - a01 * a10)**.5)


A = np.array([
              [0, 0, 0],
              [1, 0, 1],
              [0, 1, 0],
             ])

print('The surface area reads: ', surface_area(A))

### 
### Note that the non-pythonic implementation did not catch the error and gave the wrong result `area = 0.5`.
<hr style="border:1px solid blue">

### 
### `*args, **kwargs` and star syntax unpacking are among the most powerful tools for a more readable and maintainable code.
<hr style="border:1px solid blue">

### 
## <u>Lesson 3</u>: Advanced uses of Python dictionaries

In [ ]:
%reset -f

### 
### We have seen how dictionaries play an important role in handling keyword arguments.
### In what follows, we discuss some advanced uses of python dictionaries.

### 
### Python dictionaries can be utilised to avoid if-else clauses via tokenization.
### 
### <u>Task</u>:
### Write a function `solve(A, b, method='direct')` that solves $A x = b$ for $x$ using
* `method='direct'`: `sparse.linalg.spsolve`
* `method='cg'`: `sparse.linalg.cg`
* `method='gmres':` `sparse.linalg.gmres`
* `method=bicgstab:` `sparse.linalg.bicgstab`

### 
### A straightforward but cumbersome solution:

In [ ]:
from scipy.sparse import linalg as splinalg, spmatrix, diags
import numpy as np

# I see a big big big big boilerplate ...
def solve(A: spmatrix, b: np.ndarray, method: str = 'direct', **solverkwargs):
    if method == 'direct':
        print(f'Solving sparse linear system with solver method: `{method}`.')
        return splinalg.spsolve(A, b, **solverkwargs)
    elif method == 'cg':
        print(f'Solving sparse linear system with solver method: `{method}`.')
        return splinalg.cg(A, b, **solverkwargs)
    elif method == 'gmres':
        print(f'Solving sparse linear system with solver method: `{method}`.')
        return splinalg.gmres(A, b, **solverkwargs)
    elif method == 'bicgstab':
        print(f'Solving sparse linear system with solver method: `{method}`.')
        return splinalg.bicgstab(A, b, **solverkwargs)
    else:
        raise ValueError(f'Unknown method name {method}.')
        

diagonals = 2 * np.ones(10), -np.ones(9), -np.ones(9)
A = diags(diagonals, [0, -1, 1])
b = np.ones(10)

# solve with bicgstab and solver tolerance tol=1e-7
print(solve(A, b, method='bicgstab', tol=1e-7))

### 
### A pythonic solution:

In [ ]:
from scipy.sparse import linalg as splinalg, spmatrix, diags
import numpy as np

# I see only clean code =)

def solve(A: spmatrix, b: np.ndarray, method: str = 'direct', **solverkwargs):
    # Get solver from token using a dict. Return None if token is not found.
    solver = { 'bicgstab': splinalg.bicgstab,
               'direct'  : splinalg.spsolve,
               'gmres'   : splinalg.gmres, 
               'cg'      : splinalg.cg        }.get(method, None)
    
    if solver is None:  # token not found, raise error.
        raise ValueError(f'Unknown method name {method}.')
        
    print(f'Solving sparse linear system with solver method: `{method}`.')

    return solver(A, b, **solverkwargs)
        

diagonals = 2 * np.ones(10), -np.ones(9), -np.ones(9)
A = diags(diagonals, [0, -1, 1])
b = np.ones(10)

# solve with bicgstab and solver tolerance tol=1e-7
print(solve(A, b, method='bicgstab', tol=1e-7))

### 
### The `dict.get(key, default_value)` method can be used to return a default value in case a key has not been found.
<hr style="border:1px solid blue">

### 
### In the following, we discuss dict.setdefault
### Let us see what this does:

In [ ]:
test = {'a': 5, 'b': 10}

print('test: ', test, '\n')

test.setdefault('a', 20)
print('test after the first setdefault operation: ', test, '\n')

test.setdefault('c', 15)
print('test after the second setdefault operation: ', test, '\n')

### 
### The first `.setdefault` didn't do anything because the key had already been contained.
### `dict.setdefault(key, value)` only changes the `dict` if `key not in dict`
<hr style="border:1px solid blue">

### 
## <u>Exercise 3.1</u>:
### Using the `solve(A, b, ...)` function, write a function `solve_SPD(A, b, **kwargs)`
### which calls the `solve` function but uses the `cg` method unless another method token is passed.

In [ ]:
# again, the solve(...) method from above, to be used in your implementation

from scipy.sparse import linalg as splinalg, spmatrix, diags
import numpy as np


def solve(A: spmatrix, b: np.ndarray, method: str = 'direct', **solverkwargs):
    # Get solver from token using a dict. Return None if token is not found.
    solver = { 'direct'  : splinalg.spsolve,
               'bicgstab': splinalg.bicgstab,
               'gmres'   : splinalg.gmres, 
               'cg'      : splinalg.cg        }.get(method, None)
    
    if solver is None:  # token not found, raise error.
        raise ValueError(f'Unknown method name {method}.')
        
    print(f'Solving sparse linear system with solver method: `{method}`.')
    
    return solver(A, b, **solverkwargs)

In [ ]:
# incorrect solution ! this function does not have the correct signature
def solve_SPD(A, b, method='cg', **kwargs):
    return solve(A, b, method=method, **kwargs)

### 
### Your solution here:

In [ ]:
def solve_SPD(A, b, **kwargs):
    pass


diagonals = 2 * np.ones(10), -np.ones(9), -np.ones(9)
A = diags(diagonals, [0, -1, 1])
b = np.ones(10)

# solve the system but don't specify the solver, should solve with `cg`
print(solve_SPD(A, b))

# solve with `bicgstab`
print(solve_SPD(A, b, method='bicgstab'))

### 
### Solution:

In [ ]:
def solve_SPD(A, b, **kwargs):
    kwargs.setdefault('method', 'cg')
    return solve(A, b, **kwargs)


diagonals = 2 * np.ones(10), -np.ones(9), -np.ones(9)
A = diags(diagonals, [0, -1, 1])
b = np.ones(10)

# solve the system but don't specify the solver
print(solve_SPD(A, b), '\n')

# solve with bicgstab
print(solve_SPD(A, b, method='bicgstab'))

<hr style="border:1px solid blue">

### 
### By invoking `var = dict.setdefault(key, value)`, we can optionally
### capture in `var` whatever the dictionary contains at `key` after the `.setdefault` operation.

In [ ]:
test = {'a': 5, 'b': 10}

var = test.setdefault('a', 20)

print(var)

var = test.setdefault('c', 30)

print(var)

<hr style="border:1px solid blue">

### 
## <u>Exercise 3.2</u>:
### you are given a list of edges `edges = [(i0, i1), (i2, i3), ...]` representing the edges of a directed graph.
### An edge `edge = (i0, i1)` points from node `i0` to node `i1`. The nodes are not necessarily numbered from `0` to `N` but can be anything.
### Write a code that counts the number of unique edges incident to each node. Some edges may be duplicated in `edges`.
### **Remember**: you have no idea what indices the nodes have.

In [ ]:
edges = [ (1, 12), (4, 12), (4, 1), (13, 6), 
          (12, 8), (8, 4), (6, 8), (1, 12),
          (6, 4), (12, 8), (13, 6), (3, 13), (3, 4), (3, 1) ]

In [ ]:
# let's draw the graph first.
import networkx as nx
from matplotlib import pyplot as plt

G = nx.DiGraph()
G.add_edges_from(edges)

pos = nx.planar_layout(G)

nx.draw_networkx(G, arrows=True, pos=pos)
plt.show()

### 
### A crude (non-pythonic) solution:

In [ ]:
map_node_edge = {}

for edge in edges:
    # Make sure your implementation does NOT do this.
    v0 = edge[0]  # get the root node
    v1 = edge[1]  # get the incident node
    if v1 not in map_node_edge:
        map_node_edge[v1] = set()
    map_node_edge[v1].add(v0)  # add the root vertex to the set of nodes incident to i1
    
map_node_nedges = {key: len(val) for key, val in map_node_edge.items()}

print(map_node_nedges)

### 
### Your solution using dict.setdefault

In [ ]:
map_node_edge = {}

# your code here

map_node_nedges = {key: len(val) for key, val in map_node_edge.items()}

print(map_node_nedges)

### 
### solution:

In [ ]:
map_node_edge = {}

# .setdefault(key, default_value) returns whatever is at dict[key] after the .setdefault operation.
# => .setdefault(i1, set()) returns either an existing or a new set that we immediately add the root vertex to.
for v0, v1 in edges:
    map_node_edge.setdefault(v1, set()).add(v0)
    
map_node_nedges = {key: len(val) for key, val in map_node_edge.items()}

print(map_node_nedges)

<hr style="border:1px solid blue">

### 
## <u>Lesson 4</u>: Python _"truthiness"_ 

In [ ]:
%reset -f

### 
### All python built-in types (`dict`, `set`, `list`, ...) can be converted into a boolean.
### The Python developers have chosen intuitive rules as to whether an object should convert
### into `True` or `False`.
### Let's see if our intiution is correct.

### 
## <u>Exercise 4.1</u>: Does `bool(var)` convert to `True` or `False` ?

In [ ]:
# what do you think ?

objects = [ 
            None,          # untyped null pointer
            {},            # empty dict
            {1: 2},        # nonempty dict
            [],            # empty list
            [1, 2],        # nonempty list
            tuple(),       # empty tuple
            (1, 2),        # nonempty tuple
            0,             # zero integer
            0.0,           # zero float
            1.0,           # positive float
            -1.,           # negative integer
            "",            # empty string
            "connie",      # nonempty string
          ]


for var in objects:
    print('var: {}, bool(var): {}'.format(var, bool(var)))

<hr style="border:1px solid blue">


### 
### Now that we have an idea of how various variable types transform into a boolean,
### we are in the position to understand the `if other_divisors:` statement from Lesson 1.

In [ ]:
# Find the greatest divisor of an integer (excluding self) and return it.
# Also, print all other (smaller) divisors, if any.

def greatest_divisor_excluding_self(val):
    """ excluding `val` itself """
    *other_divisors, greatest_divisor = [i for i in range(1, val) if val % i == 0]
    if other_divisors:
        print("For {}, I also found the divisors {}.".format(val, other_divisors))
    return greatest_divisor

print('The greatest divisor (excl self) of 10 equals:', greatest_divisor_excluding_self(10))

### 
### First, we break down the line `*other_divisors, greatest_divisor = [i for i in range(1, val) if val % i == 0]`
* the rhs creates a monotone increasing list with at least one element `1`.
* the left hand side peels off the last (and hence largest) element in that list, while collecting all remaining divisors in the list `other_divisors`
* if the only divisor is `[1]`, `other_divisors == []`, else `other_divisors != []`.

### Then, the line `if other_divisors:`
* in the next line `if other_divisors:`, Python, under the hood, converts this line to `if bool(other_divisors):`
* from before, we know that if `other_divisors == []`, `if other_divisors:` becomes `if False:` and the if clause is ignored.
* On the other hand, if `other_divisors != []` (nonempty), then `if other_divisors:` converts to `if True:` and the if clause's code is executed.
<hr style="border:1px solid blue">

### 
## <u>Exercise 4.2</u>:
### write a function `average(*list_of_numbers)` that computes the average of the numbers in `list_of_numbers` and returns `0` if no numbers have been passed.
### Use Python truthiness !

In [ ]:
def average(*list_of_numbers):
    # your code here
    pass


print(average(1, 2, 3))
print(average())

### 
### 2 equivalent solutions:

In [ ]:
def average_v0(*list_of_numbers):
    if list_of_numbers:
        # note that if list_of_numbers == [], we divide by 0.
        return sum(list_of_numbers) / len(list_of_numbers)
    return 0
    
    
print(average_v0(1, 2, 3))
print(average_v0())

# one liner
def average_v1(*list_of_numbers):
    return sum(list_of_numbers) / len(list_of_numbers) if list_of_numbers else 0
    
    
print(average_v1(1, 2, 3))
print(average_v1())